In [0]:
'''
Gensim Doc2Vec
'''
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import random
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import gensim
import sklearn.metrics

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Load Data
SOS_token = 0
EOS_token = 1
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

import pickle
with open('/content/drive/My Drive/CS 263 Project/Final Project Data/baking_data_title_ingredients.pickle','rb') as f:
    baking_data = pickle.load(f)

cleaned_recipes = baking_data[1]
cleaned_recipes_IDs = baking_data[9]

In [0]:
#limit to length 75
MAX_LENGTH = 75
cleaned_recipes_IDs = [cleaned_recipes_IDs[i] for i in range(0,len(cleaned_recipes)) if len(cleaned_recipes[i].split())<= MAX_LENGTH]
cleaned_recipes = [cleaned_recipes[i] for i in range(0,len(cleaned_recipes)) if len(cleaned_recipes[i].split())<= MAX_LENGTH]
len(cleaned_recipes_IDs)
len(cleaned_recipes)

In [0]:
#process Text
def create_corpus(dta, tokens_only=False):
    for i, line in enumerate(dta):
        tokens = line
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

corpus = list(create_corpus(cleaned_recipes))
#test_corpus = list(create_corpus(cleaned_recipes[100000:], tokens_only=True))
corpus2 = list(create_corpus(cleaned_recipes, tokens_only=True))

In [0]:
#define model, get vocab, and train model
hidden_size = 512
model = gensim.models.doc2vec.Doc2Vec(vector_size=hidden_size, min_count=1, epochs=20)

In [0]:
model.build_vocab(corpus)

In [0]:
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [0]:
#Save model
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/content/drive/My Drive/CS 263 Project/Saved Models/doc2vec_model")
model.save(fname)

In [0]:
#load model
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/content/drive/My Drive/CS 263 Project/Saved Models/doc2vec_model")
model = gensim.models.doc2vec.Doc2Vec.load(fname)

In [0]:
#get embeddings
import numpy as np
embeddings = np.empty([len(corpus),hidden_size])
for i in range(len(corpus)):
  embeddings[i,:] = model.infer_vector(corpus2[i])
embeddings.shape

In [0]:
#get embeddings for recipies with nutritional info
with open('/content/drive/My Drive/CS 263 Project/Final Project Data/nutritional_info.pickle','rb') as f:
    nutritional_info = pickle.load(f)

embeddings_IDs = [cleaned_recipes_IDs[i] for i in range(0,len(cleaned_recipes_IDs)) if len(cleaned_recipes[i].split())<= MAX_LENGTH]
nutri_embeddings_indices = [i for i, val in enumerate(embeddings_IDs) if val in nutritional_info["id"].tolist()]

nutri_embeddings_IDs = [embeddings_IDs[i] for i in nutri_embeddings_indices]
nutri_embeddings = [embeddings[i] for i in nutri_embeddings_indices]

In [0]:
#similarity matrix
cosine_similarities = sklearn.metrics.pairwise.cosine_similarity(nutri_embeddings, Y=None, dense_output=False)
print(cosine_similarities)
print(np.isnan(cosine_similarities).any())
print(np.isfinite(cosine_similarities).any())

In [0]:
import pickle
similarity_pickle = [nutri_embeddings,nutri_embeddings_IDs,cosine_similarities]
with open('/content/drive/My Drive/CS 263 Project/Final Project Data/doc2vec_similarity.pickle', 'wb') as f:
    pickle.dump(similarity_pickle, f)